In [148]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from datasets import load_dataset
import numpy as np
import time
import torch
from sklearn.metrics import f1_score
import pandas as pd

In [149]:
go_emotions = load_dataset('go_emotions', split='train')
go_emotions

Dataset({
    features: ['text', 'labels', 'id'],
    num_rows: 43410
})

In [150]:

# GoEmotions эмоции (28 штук)
goemotions_emotions = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 
    'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 
    'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 
    'high-spirited', 'hope', 'joy', 'love', 'nervousness', 'optimism', 
    'pride', 'realization', 'relief', 'sadness', 'surprise', 'neutral'
]

# Общие с моделями (5 эмоций)
common_emotions = ['anger', 'fear', 'joy', 'sadness', 'surprise']

# Функция фильтра: оставляем только сэмплы с хотя бы одной общей эмоцией
def filter_common_emotions(example):
     # Проверяем каждую эмоцию из labels
    for label_idx in example['labels']:
        emotion = goemotions_emotions[label_idx]
        if emotion in common_emotions:
            return True
    return False

# Фильтрация датасета
filtered_dataset = go_emotions.filter(filter_common_emotions)

In [151]:
true_pred = [[goemotions_emotions[i] for i in label_idx] for label_idx in filtered_dataset['labels']]
true_pred

[['surprise'],
 ['surprise'],
 ['anger'],
 ['fear'],
 ['sadness'],
 ['surprise'],
 ['surprise'],
 ['anger'],
 ['surprise'],
 ['confusion', 'disappointment', 'surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['anger'],
 ['surprise'],
 ['surprise'],
 ['anger'],
 ['surprise'],
 ['anger', 'curiosity'],
 ['surprise'],
 ['surprise'],
 ['grief', 'surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['anger'],
 ['anger'],
 ['surprise'],
 ['surprise'],
 ['disappointment', 'surprise'],
 ['surprise'],
 ['anger', 'annoyance'],
 ['sadness'],
 ['anger'],
 ['s

In [152]:
base = pipeline("text-classification", model="bhadresh-savani/roberta-base-emotion", return_all_scores=True)
large1 = pipeline("text-classification", model="j-hartmann/emotion-english-roberta-large")
large2 = pipeline("text-classification", model="cardiffnlp/twitter-roberta-large-emotion-latest")

Device set to use cuda:0
c:\Users\Vlad\OneDrive\Рабочий стол\pet\explaratory_data_analysis\.venv\Lib\site-packages\transformers\pipelines\text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Device set to use cuda:0
Device set to use cuda:0


In [ ]:
def predict_labels_base(model, texts):
    preds = []
    start = time.time()
    
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats(0)  # device=0
        torch.cuda.synchronize()  # ждём завершения CUDA
        vram_base = torch.cuda.memory_allocated(0) / 1024**3
    
    for text in texts:
        score = model(text, truncation=True, max_length=512)
        pred_label = max(score[0], key=lambda x: x['score'])['label']
        preds.append(pred_label)
    
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        vram_peak = torch.cuda.max_memory_allocated(0) / 1024**3  # ← ПИК!
        vram_delta = vram_peak - vram_base
    
    end = time.time()
    
    return preds, end - start, vram_delta

def predict_labels_large(model, texts):
    preds = []
    start = time.time()
        
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats(0)  # device=0
        torch.cuda.synchronize()  # ждём завершения CUDA
        vram_base = torch.cuda.memory_allocated(0) / 1024**3

    for text in texts:
        score = model(text, truncation=True, max_length=512)
        pred_label = max(score, key=lambda x: x['score'])['label']
        preds.append(pred_label)

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        vram_peak = torch.cuda.max_memory_allocated(0) / 1024**3  # ← ПИК!
        vram_delta = vram_peak - vram_base

    end = time.time()
    vram_end = torch.cuda.memory_allocated(0) / 1024**3 if torch.cuda.is_available() else 0

    return preds, end - start, vram_delta

In [154]:
texts = list(filtered_dataset['text'])[:10000]
predicted_base, time_base, vram_base = predict_labels_base(base, texts)
predicted_large1, time_large1, vram_large1 = predict_labels_large(large1, texts)
predicted_large2, time_large2, vram_large2 = predict_labels_large(large2, texts)

In [155]:
def check_prediction_in_true(single_pred, true_labels_list):
    """1 если предсказание в true labels, иначе 0"""
    return 1 if single_pred in true_labels_list else 0

In [156]:
predicted_base_hitrate = [check_prediction_in_true(pred, true_pred[i]) for i, pred in enumerate(predicted_base)]
predicted_large1_hitrate = [check_prediction_in_true(pred, true_pred[i]) for i, pred in enumerate(predicted_large1)]
predicted_large2_hitrate = [check_prediction_in_true(pred, true_pred[i]) for i, pred in enumerate(predicted_large2)]
true_labels = np.ones(10000)

In [157]:
f1_base = f1_score(predicted_base_hitrate, true_labels)
f1_large1 = f1_score(predicted_large1_hitrate, true_labels)
f1_large2 = f1_score(predicted_large2_hitrate, true_labels)

In [158]:
result = pd.DataFrame({'models': ['roberta-base-emotion', 'emotion-english-roberta-large', 'twitter-roberta-base-emotion'],
                       'f1_score': [f1_base, f1_large1, f1_large2],
                       'time(1000 exaples)': [time_base, time_large1, time_large2],
                       'vram': [vram_base, vram_large1, vram_large2]})
result

,models,f1_score,time(1000 exaples),vram
0,roberta-base-emotion,0.173349,83.114247,0.016853
1,emotion-english-roberta-large,0.291183,244.557366,0.021492
2,twitter-roberta-base-emotion,0.208848,145.007240,0.021492
